In [1]:
import numpy as np
from scipy.stats import norm

from src.cython_core import sample_sov
from src.utils import ci_bisection

Suppose we observe $Z\in \mathbb{R}^d$ which follows the distribution
$$
\begin{pmatrix} Z \\ \omega \end{pmatrix} \sim N_{2d}\left(\begin{pmatrix} \mu \\ 0_d \end{pmatrix}, \begin{pmatrix} \Sigma & 0 \\ 0 & \Omega\end{pmatrix} \right) |_{A(Z+\omega)\leq b }.
$$
We want to find a p-value for $H_0:\eta^\intercal \mu=\theta$.

Denote $\nu=\eta^\intercal \Sigma \eta$, $c=\nu^{-1} \Sigma \eta$, $\hat\theta=\eta^\intercal Z$.
Write $Z=c \hat\theta + (I - c\eta^\intercal)Z$. Conditional on $(I - c\eta^\intercal)Z=u$, we have under $H_0$
$$
\begin{pmatrix} \hat\theta \\ \omega \end{pmatrix} \sim N\left(\begin{pmatrix} \theta \\ 0 \end{pmatrix}, \begin{pmatrix} \nu & 0 \\ 0 & \Omega\end{pmatrix} \right) |_{A(c \hat\theta + u +\omega)\leq b }.
$$

Define $Y=A(c \hat\theta + u + \omega)$. We have
$$
\begin{pmatrix} \hat\theta \\ Y \end{pmatrix} \sim N\left(\begin{pmatrix} \theta \\ A(c\theta+u) \end{pmatrix}, \begin{pmatrix} \nu & (Ac\nu)^\intercal \\ Ac\nu & A(\nu cc^\intercal + \Omega)A^\intercal \end{pmatrix} \right) |_{Y\leq b }.
$$
The conditional distribution of $\hat\theta\mid Y$ is $N( m(Y), \tau^2)$, where
$$
m(Y)=\theta + (Ac\nu)^\intercal (A(\nu cc^\intercal + \Omega)A^\intercal )^{-1}(Y - A(c\theta+u) ),\quad \tau^2=\nu - (Ac\nu)^\intercal (A(\nu cc^\intercal + \Omega)A^\intercal )^{-1} (Ac\nu)
$$

The CDF of $\hat\theta$ is
$$
P(\hat\theta \leq t) = \frac{\int_{Y\leq b} \Phi(\frac{t-m}{\tau}) \varphi(Y; A(c\theta+u), A(\nu cc^\intercal+\Omega)A^\intercal )   dY}{\int_{Y\leq b} \varphi(Y; A(c\theta+u), A(\nu cc^\intercal+\Omega)A^\intercal )   dY  }.
$$

The integrals in the numerator and denominator are evaluated by Monte Carlo.
We apply the separation-of-variable (SOV) to sample $Y$ from the truncated normal distribution. The SOV method is essentially an importance sampling method. If we get $N$ samples $(Y_i,w_i)$ where $w_i$ is the associated importance weight, then the CDF $P(\hat\theta\leq t)$ is estimated by
$$
\hat F(t)= \frac{\sum_{i=1}^N w_i \Phi(\frac{t-m(Y_i)}{\tau} ) }{\sum_{i=1}^N w_i}.
$$
A two-sided p-value is obtained by $2 \min(\hat F(t), 1-\hat F(t) )$.

Confidence intervals can be obtained by inverting the test. We do this by a bi-section method.

In [2]:
# generate Z
d = 5
mu = np.zeros(d) - .5
Sigma = np.eye(d) * .5 + .5
Omega = .5 * np.eye(d)
A = np.tril(.5 * np.ones((d, d)))
b = np.zeros(d)

def generate_data(rng):
    while True:
        z = rng.multivariate_normal(mu, Sigma)
        omega = rng.multivariate_normal(np.zeros(d), Omega)
        if np.all(A @ (z + omega) <= b):
            return z, omega

rng = np.random.default_rng(0)
Z, _ = generate_data(rng)

In [3]:
eta = np.eye(d)[0]
nu = np.dot(eta, Sigma @ eta)
c = Sigma @ eta
theta_hat = eta.T @ Z
u = Z - c * theta_hat

cov_Y = A @ (nu * np.outer(c, c) + Omega) @ A.T
theta_hat_conditional_sd = np.sqrt(nu - (A @ c * nu).T @ np.linalg.inv(cov_Y) @ (A @ c * nu))

In [4]:
def sample_trunc_normal(m, Sigma, seed=1, nsample=512):
    """"
    sample from the truncated normal distribution X ~ N(m, Sigma) | X < 0
    m: mean vector
    Sigma: covariance matrix
    seed: seed
    nsample: number of Monte Carlo samples
    """
    L = np.linalg.cholesky(Sigma)
    lower_limit = -np.inf * np.ones_like(m)
    upper_limit = -m
    samples, weights = sample_sov(lower_limit, upper_limit, L, nsample, seed, spherical=0)
    samples = samples + m
    return samples, weights

In [5]:
def get_pvalue(theta, two_sided=True):
    """
    Compute the p-value under eta' mu = theta
    """
    # sample Y from the truncated normal distribution
    mean_Y = A @ (c * theta + u)
    
    # take 512 weighted samples by QMC (the sample size must be powers of 2)
    Y_samples, weights = sample_trunc_normal(mean_Y, cov_Y, seed=1, nsample=512) 

    # compute the conditional mean and standard deviation of \hat{theta} given Y
    theta_hat_conditional_mean = theta + (A @ c * nu).T @ np.linalg.inv(cov_Y) @ (Y_samples - mean_Y).T

    # compute the CDF of \hat{\theta} evaluated at the observed theta_hat; this is the importance sampling estimator
    pval = np.mean(norm.cdf((theta_hat - theta_hat_conditional_mean) / theta_hat_conditional_sd) * weights) / np.mean(weights)

    if two_sided:
        pval = 2 * min(pval, 1 - pval)
    
    return pval


In [6]:
pvalue = get_pvalue(0)
print("P-value for eta'mu = 0:", np.round(pvalue, 4))

P-value for eta'mu = 0: 0.2082


In [7]:
ci = ci_bisection(get_pvalue, theta_hat, theta_hat_conditional_sd, 0.05)
print("Interval for eta'mu:", np.round(ci, 4))

Interval for eta'mu: [-1.05   6.188]


In [8]:
print("True value of eta'mu:", eta.dot(mu))

True value of eta'mu: -0.5


In [9]:
# simulation
from tqdm import trange

d = 5
mu = np.zeros(d) - .5
Sigma = np.eye(d) * .5 + .5
Omega = .5 * np.eye(d)
A = np.tril(.5 * np.ones((d, d)))
b = np.zeros(d)
eta = np.eye(d)[0]
true_theta = eta.dot(mu)
nu = np.dot(eta, Sigma @ eta)
c = Sigma @ eta
cov_Y = A @ (nu * np.outer(c, c) + Omega) @ A.T
theta_hat_conditional_sd = np.sqrt(nu - (A @ c * nu).T @ np.linalg.inv(cov_Y) @ (A @ c * nu))
sample_splitting_sd = np.sqrt(nu + np.dot(Sigma @ eta, np.linalg.inv(Omega) @ Sigma @ eta))

def generate_data(rng):
    while True:
        z = rng.multivariate_normal(mu, Sigma)
        omega = rng.multivariate_normal(np.zeros(d), Omega)
        if np.all(A @ (z + omega) <= b):
            return z, omega
            
def simu(rng):
    Z, omega = generate_data(rng)
    theta_hat = eta.T @ Z
    u = Z - c * theta_hat
    pval_conditional = get_pvalue(theta_hat)
    ci_conditional = ci_bisection(get_pvalue, theta_hat, theta_hat_conditional_sd, 0.05)
    covered_conditional = ci[0] <= true_theta <= ci[1]

    # "sample splitting"
    Z_indep = Z - Sigma @ np.linalg.inv(Omega) @ omega
    pval_splitting = norm.cdf(eta.dot(Z_indep), loc=true_theta, scale=sample_splitting_sd)
    ci_splitting = eta.dot(Z_indep) + np.array([-1, 1]) * norm.ppf(0.975) * sample_splitting_sd
    covered_splitting = ci_splitting[0] <= true_theta <= ci_splitting[1]
    return pval_conditional, ci_conditional, covered_conditional, pval_splitting, ci_splitting, covered_splitting

pvals = {'conditional': [], 'splitting': []}
lens = {'conditional': [], 'splitting': []}
covered = {'conditional': [], 'splitting': []}
for _ in trange(100):
    pval_conditional, ci_conditional, covered_conditional, pval_splitting, ci_splitting, covered_splitting = simu(rng)
    
    pvals['conditional'].append(pval_conditional)
    pvals['splitting'].append(pval_splitting)
    lens['conditional'].append(ci_conditional[1] - ci_conditional[0])
    lens['splitting'].append(ci_splitting[1] - ci_splitting[0])
    covered['conditional'].append(covered_conditional)
    covered['splitting'].append(covered_splitting)

100%|██████████| 100/100 [00:33<00:00,  3.02it/s]


In [10]:
import pandas as pd

print('Coverage proportions:')
pd.DataFrame(covered).mean()

Coverage proportions:


conditional    1.00
splitting      0.97
dtype: float64

In [11]:
print('Average interval lengths:')
pd.DataFrame(lens).mean()

Average interval lengths:


conditional    6.804584
splitting      8.765225
dtype: float64